# Analysis of Chinese economic trade

## Import Libraries

In [2]:
import pandas as pd
import numpy as np
pd.options.display.max_columns=None
pd.options.display.max_rows=None
import matplotlib.pyplot as plt

## Read data

In [213]:
#Load main dataset
df_hs = pd.read_csv('country_partner_hsproduct2digit_year.csv')

In [4]:
#Load dataset with product metadata
df_product = pd.read_csv('hs_product.tab', sep='\t')

In [13]:
df_loc = pd.read_csv('location.tab', sep='\t')

In [142]:
#Inspect number of rows and columns
df_hs.shape

(23529052, 11)

In [6]:
#Get column names
df_hs.columns

Index(['location_id', 'partner_id', 'product_id', 'year', 'export_value',
       'import_value', 'hs_eci', 'hs_coi', 'location_code', 'partner_code',
       'hs_product_code'],
      dtype='object')

In [163]:
#Quick data check of main dataset
df_hs.head(20)

,location_id,partner_id,product_id,year,export_value,import_value,hs_eci,hs_coi,location_code,partner_code,hs_product_code
0,0,1,103,2018,27561,0,0.885589,0.124824,ABW,AFG,4
1,0,1,103,2019,40943,0,1.030314,-0.021657,ABW,AFG,4
2,7,1,103,2013,5550163,7303,-0.070443,-0.585271,ARE,AFG,4
3,7,1,103,2019,3956845,261,-0.177291,-0.864360,ARE,AFG,4
4,7,1,103,2018,4577937,0,0.099466,-0.345645,ARE,AFG,4
5,7,1,103,2017,1336541,6828,0.218627,0.019633,ARE,AFG,4
6,7,1,103,2014,9521655,3987,0.095593,-0.296020,ARE,AFG,4
7,7,1,103,2016,1765694,4584,0.108619,0.043275,ARE,AFG,4
8,7,1,103,2012,1469891,0,0.451211,-0.284264,ARE,AFG,4
9,7,1,103,2008,950950,0,-0.276794,-0.691036,ARE,AFG,4


In [214]:
#Quick inspection of product dataset
df_product.head(10)

,product_id,hs_product_code,hs_product_name_short_en,level,parent_id
0,0,0,Textiles,section,NaN
1,1,1,Agriculture,section,NaN
2,2,2,Stone,section,NaN
3,3,3,Minerals,section,NaN
4,4,4,Metals,section,NaN
5,5,5,Chemicals,section,NaN
6,6,6,Vehicles,section,NaN
7,7,7,Machinery,section,NaN
8,8,8,Electronics,section,NaN
9,9,9,Other,section,NaN


In [169]:
#Create a subset for 2digit-level products
df_product_2digit = df_product.loc[df_product.level=='2digit']

In [182]:
#Review datatype
df_product_2digit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 11 to 112
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   product_id                102 non-null    int64 
 1   hs_product_code           102 non-null    object
 2   hs_product_name_short_en  102 non-null    object
 3   level                     102 non-null    object
 4   parent_id                 102 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.8+ KB


In [171]:
df_product_section = df_product.loc[df_product.level=='section']

In [192]:
df_product_section.head()

,product_id,hs_product_code,hs_product_name_short_en,level,parent_id
0,0,0,Textiles,section,nan
1,1,1,Agriculture,section,nan
2,2,2,Stone,section,nan
3,3,3,Minerals,section,nan
4,4,4,Metals,section,nan


In [193]:
df_product_2digit['parent_id'] = pd.to_numeric(df_product_2digit["parent_id"])


In [207]:
df_product_2digit_expanded = df_product_2digit.merge(df_product_section, how='left', left_on='parent_id', right_on='product_id', suffixes=('','_section'))

In [211]:
df_product_2digit_expanded.head(5)

,product_id,hs_product_code,hs_product_name_short_en,parent_id,hs_product_name_short_en_section
0,100,01,Live animals,1.0,Agriculture
1,101,02,Meat,1.0,Agriculture
2,102,03,Fish,1.0,Agriculture
3,103,04,Diary products,1.0,Agriculture
4,104,05,Animal products,1.0,Agriculture


In [204]:
df_product_2digit_expanded.columns

Index(['product_id', 'hs_product_code', 'hs_product_name_short_en', 'level',
       'parent_id', 'product_id_section', 'hs_product_code_section',
       'hs_product_name_short_en_section', 'level_section',
       'parent_id_section'],
      dtype='object')

In [208]:
df_product_2digit_expanded.drop(['level', 'product_id_section', 'hs_product_code_section',
       'level_section','parent_id_section'], axis=1, inplace=True)

In [14]:
df_loc.shape

(258, 5)

In [15]:
df_loc.head(10)

,location_id,location_code,location_name_short_en,level,parent_id
0,0,ABW,Aruba,country,356.0
1,1,AFG,Afghanistan,country,353.0
2,2,AGO,Angola,country,352.0
3,3,AIA,Anguilla,country,356.0
4,4,ALB,Albania,country,355.0
5,5,AND,Andorra,country,355.0
6,6,ANT,Netherlands Antilles,country,356.0
7,7,ARE,United Arab Emirates,country,353.0
8,8,ARG,Argentina,country,357.0
9,9,ARM,Armenia,country,353.0


In [15]:
df_loc[df_loc.parent_id==352].head()

,location_id,location_code,location_name_short_en,level,parent_id
2,2,AGO,Angola,country,352.0
17,17,BDI,Burundi,country,352.0
19,19,BEN,Benin,country,352.0
21,21,BFA,Burkina Faso,country,352.0
37,37,BWA,Botswana,country,352.0


In [16]:
df_hs.head()

,location_id,partner_id,product_id,year,export_value,import_value,hs_eci,hs_coi,location_code,partner_code,hs_product_code
0,0,1,103,2018,27561,0,0.885589,0.124824,ABW,AFG,4
1,0,1,103,2019,40943,0,1.030314,-0.021657,ABW,AFG,4
2,7,1,103,2013,5550163,7303,-0.070443,-0.585271,ARE,AFG,4
3,7,1,103,2019,3956845,261,-0.177291,-0.864360,ARE,AFG,4
4,7,1,103,2018,4577937,0,0.099466,-0.345645,ARE,AFG,4


In [17]:
df_loc.columns

Index(['location_id', 'location_code', 'location_name_short_en', 'level',
       'parent_id'],
      dtype='object')

In [17]:
#Create a copy of the main dataset
df = df_hs.copy()

In [212]:
df.columns

Index(['location_id', 'partner_id', 'product_id', 'year', 'export_value',
       'import_value', 'hs_eci', 'hs_coi', 'location_code', 'partner_code',
       'hs_product_code'],
      dtype='object')

In [18]:
df_short_loc = df_loc[['location_id', 'location_name_short_en', 'parent_id']]

In [19]:
df1 = df.merge(df_short_loc, how='left', on='location_id')

In [20]:
df1.shape

(23529052, 13)

In [21]:
df2 = df1.merge(df_short_loc, how='left', left_on='partner_id', right_on='location_id', suffixes=('', '_right'))

In [22]:
df2.drop('location_id_right', axis=1, inplace=True)

In [23]:
df2.columns

Index(['location_id_left', 'partner_id', 'product_id', 'year', 'export_value',
       'import_value', 'hs_eci', 'hs_coi', 'location_code', 'partner_code',
       'hs_product_code', 'location_name_short_en_left', 'parent_id_left',
       'location_name_short_en_right', 'parent_id_right'],
      dtype='object')

In [24]:
df2.rename(columns={"location_id_left":'location_id','location_name_short_en_left':'location_name', 'location_name_short_en_right':'partner_name', 'parent_id_right':'region_id'}, inplace=True)
                    

In [25]:
df2.shape

(23529052, 15)

In [ ]:
china = df2[df2.location_code=='CHN']

In [ ]:
china[china['partner_code']=='DEU'][['partner_name', 'year','export_value']].groupby(['partner_name', 'year'], as_index=False).sum().sort_values(by=['partner_name', 'year'])

In [26]:
df3 = df2[['location_name','partner_name', 'year', 'export_value']].groupby(['location_name','partner_name', 'year'], as_index=False).sum().sort_values(by=['location_name','partner_name', 'year'])


In [ ]:
temp_sum = china.groupby(['partner_code', 'year'])['export_value'].sum()

In [ ]:
df3 = temp_sum.groupby(level=0).apply(lambda x: 100*x/float(x.sum()))

In [27]:
df3.to_csv('datafile.csv')

In [ ]:
df_new = pd.DataFrame(df3)

In [ ]:
df_new.to_csv('china_file.csv')

In [ ]:
result = d.groupby(level=0).apply(lambda x: 100*x/float(x.sum()))

In [ ]:
percent_total_.head(20)

In [ ]:
percent_total_.shape

In [35]:
df2.shape

(23529052, 15)

In [ ]:
df2['new'] = df2.groupby(['location_name'])

In [31]:
df_new = df2.groupby(['location_name', 'year'])['partner_name'].nunique()
df_new = df_new.to_frame(name = 'num_partners_by_year').reset_index()

In [32]:
df_new.head()

,location_name,year,num_partners_by_year
0,Afghanistan,1995,50
1,Afghanistan,1996,58
2,Afghanistan,1997,58
3,Afghanistan,1998,59
4,Afghanistan,1999,61


In [36]:
df2.columns

Index(['location_id', 'partner_id', 'product_id', 'year', 'export_value',
       'import_value', 'hs_eci', 'hs_coi', 'location_code', 'partner_code',
       'hs_product_code', 'location_name', 'parent_id_left', 'partner_name',
       'region_id'],
      dtype='object')

In [81]:
grouped_df = df2[['year', 'export_value',  'import_value', 'location_name', 'partner_name']].groupby(['year','location_name', 'partner_name'], as_index=False).sum().sort_values(by=['location_name','partner_name', 'year'])


In [82]:
grouped_df.shape

(622026, 5)

In [47]:
export_df =df2[['year', 'export_value',  'location_name', 'partner_name']].groupby(['year','location_name', 'partner_name'], as_index=False).sum().sort_values(by=['location_name','partner_name', 'year'])



In [51]:
export_df.shape

(517987, 4)

In [50]:
export_df = export_df[export_df.export_value>0]

In [52]:
#create variable for the number of unique partners by location name and year
df3 = export_df.groupby(['location_name', 'year'])['partner_name'].nunique()
df3 = df3.to_frame(name = 'num_partners_by_year').reset_index()

In [53]:
df3.head()

,location_name,year,num_partners_by_year
0,Afghanistan,1995,36
1,Afghanistan,1996,40
2,Afghanistan,1997,47
3,Afghanistan,1998,42
4,Afghanistan,1999,45


In [62]:
df3_dict = {(df3.location_name[x],df3.year[x]):df3.num_partners_by_year[x] for x in range(len(df3))}

In [63]:
df3_dict

{('Afghanistan', 1995): 36,
 ('Afghanistan', 1996): 40,
 ('Afghanistan', 1997): 47,
 ('Afghanistan', 1998): 42,
 ('Afghanistan', 1999): 45,
 ('Afghanistan', 2000): 47,
 ('Afghanistan', 2001): 40,
 ('Afghanistan', 2002): 42,
 ('Afghanistan', 2003): 53,
 ('Afghanistan', 2004): 49,
 ('Afghanistan', 2005): 52,
 ('Afghanistan', 2006): 50,
 ('Afghanistan', 2007): 55,
 ('Afghanistan', 2008): 61,
 ('Afghanistan', 2009): 55,
 ('Afghanistan', 2010): 61,
 ('Afghanistan', 2011): 57,
 ('Afghanistan', 2012): 52,
 ('Afghanistan', 2013): 63,
 ('Afghanistan', 2014): 57,
 ('Afghanistan', 2015): 54,
 ('Afghanistan', 2016): 53,
 ('Afghanistan', 2017): 52,
 ('Afghanistan', 2018): 63,
 ('Afghanistan', 2019): 63,
 ('Afghanistan', 2020): 42,
 ('Albania', 1995): 33,
 ('Albania', 1996): 35,
 ('Albania', 1997): 32,
 ('Albania', 1998): 32,
 ('Albania', 1999): 32,
 ('Albania', 2000): 33,
 ('Albania', 2001): 36,
 ('Albania', 2002): 38,
 ('Albania', 2003): 44,
 ('Albania', 2004): 48,
 ('Albania', 2005): 54,
 ('Alban

In [83]:
grouped_df['num_export_partners_by_country_year'] = [df3_dict.get((grouped_df.location_name[x],grouped_df.year[x]),0) for x in range(len(grouped_df))]

In [54]:
import_df =df2[['year', 'import_value',  'location_name', 'partner_name']].groupby(['year','location_name', 'partner_name'], as_index=False).sum().sort_values(by=['location_name','partner_name', 'year'])



In [57]:
import_df.shape

(517987, 4)

In [56]:
import_df = import_df[import_df.import_value>0]

In [58]:
#create variable for the number of unique partners by location name and year
df4 = import_df.groupby(['location_name', 'year'])['partner_name'].nunique()
df4 = df4.to_frame(name = 'num_partners_by_year').reset_index()

In [59]:
df4.head()

,location_name,year,num_partners_by_year
0,Afghanistan,1995,36
1,Afghanistan,1996,43
2,Afghanistan,1997,42
3,Afghanistan,1998,47
4,Afghanistan,1999,45


In [64]:
df4_dict = {(df4.location_name[x],df4.year[x]):df4.num_partners_by_year[x] for x in range(len(df4))}

In [65]:
df4_dict

{('Afghanistan', 1995): 36,
 ('Afghanistan', 1996): 43,
 ('Afghanistan', 1997): 42,
 ('Afghanistan', 1998): 47,
 ('Afghanistan', 1999): 45,
 ('Afghanistan', 2000): 51,
 ('Afghanistan', 2001): 48,
 ('Afghanistan', 2002): 57,
 ('Afghanistan', 2003): 65,
 ('Afghanistan', 2004): 67,
 ('Afghanistan', 2005): 72,
 ('Afghanistan', 2006): 71,
 ('Afghanistan', 2007): 75,
 ('Afghanistan', 2008): 85,
 ('Afghanistan', 2009): 91,
 ('Afghanistan', 2010): 92,
 ('Afghanistan', 2011): 87,
 ('Afghanistan', 2012): 81,
 ('Afghanistan', 2013): 83,
 ('Afghanistan', 2014): 75,
 ('Afghanistan', 2015): 89,
 ('Afghanistan', 2016): 81,
 ('Afghanistan', 2017): 85,
 ('Afghanistan', 2018): 90,
 ('Afghanistan', 2019): 88,
 ('Afghanistan', 2020): 59,
 ('Albania', 1995): 42,
 ('Albania', 1996): 52,
 ('Albania', 1997): 53,
 ('Albania', 1998): 51,
 ('Albania', 1999): 68,
 ('Albania', 2000): 71,
 ('Albania', 2001): 68,
 ('Albania', 2002): 78,
 ('Albania', 2003): 89,
 ('Albania', 2004): 84,
 ('Albania', 2005): 85,
 ('Alban

In [84]:
grouped_df['num_import_partners_by_country_year'] = [df4_dict.get((grouped_df.location_name[x],grouped_df.year[x]),0) for x in range(len(grouped_df))]

In [85]:
grouped_df['total_export_by_location_year'] = grouped_df.groupby(['location_name', 'year'])['export_value'].transform('sum')

In [119]:
grouped_df['total_import_by_location_year'] = grouped_df.groupby(['location_name', 'year'])['import_value'].transform('sum')

In [86]:
grouped_df.head()

,year,location_name,partner_name,export_value,import_value,num_export_partners_by_country_year,num_import_partners_by_country_year,total_export_by_location_year
257328,2007,Afghanistan,Albania,0,2057713,36,36,372909932
360830,2011,Afghanistan,Albania,0,106445,36,36,523362598
17672,1996,Afghanistan,Algeria,212007,0,36,36,83146051
36008,1997,Afghanistan,Algeria,181929,0,36,36,103148983
55224,1998,Afghanistan,Algeria,383822,0,36,36,120012405


In [87]:
grouped_df['share'] = [grouped_df.export_value[x]/grouped_df.total_export_by_location_year[x] for x in range(len(grouped_df))]

/Users/jorgemirandamontano/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in long_scalars
  """Entry point for launching an IPython kernel.


In [123]:
grouped_df = grouped_df.assign(proportion_export_year=lambda x: (x['export_value']/x['total_export_by_location_year'])*100)

In [121]:
grouped_df = grouped_df.assign(proportion_import_year=lambda x: (x['import_value']/x['total_import_by_location_year'])*100)

In [75]:
df2_copy = df2.copy()

In [76]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23529052 entries, 0 to 23529051
Data columns (total 20 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   location_id                          int64  
 1   partner_id                           int64  
 2   product_id                           int64  
 3   year                                 int64  
 4   export_value                         int64  
 5   import_value                         int64  
 6   hs_eci                               float64
 7   hs_coi                               float64
 8   location_code                        object 
 9   partner_code                         object 
 10  hs_product_code                      int64  
 11  location_name                        object 
 12  parent_id_left                       float64
 13  partner_name                         object 
 14  region_id                            float64
 15  num_export_partners_by_country

In [124]:
grouped_df['rank_export_year'] = grouped_df.groupby(['location_name', 'year'])['proportion_export_year'].rank('dense', ascending = False)


In [125]:
grouped_df['rank_import_year'] = grouped_df.groupby(['location_name', 'year'])['proportion_import_year'].rank('dense', ascending = False)


In [128]:
grouped_df[(grouped_df.location_name == 'China') & (grouped_df.year== 2019)].sort_values(by='export_value', ascending=False)

,year,location_name,partner_name,export_value,import_value,num_export_partners_by_country_year,num_import_partners_by_country_year,total_export_by_location_year,share,rank,proportion,total_import_by_location_year,proportion_import_year,proportion_export_year,rank_export_year,rank_import_year
579291,2019,China,United States of America,387348498338,105880174597,205,166,2455821664459,0.000322,1.0,15.772664,1643417403153,6.442683,15.772664,1.0,5.0
579173,2019,China,Hong Kong,278149269736,17387935826,104,95,2455821664459,0.002764,2.0,11.326118,1643417403153,1.058035,11.326118,2.0,24.0
579184,2019,China,Japan,145552998143,141336673954,117,96,2455821664459,0.000712,3.0,5.926855,1643417403153,8.600169,5.926855,3.0,2.0
579263,2019,China,South Korea,110378926508,143445066635,200,158,2455821664459,0.000035,4.0,4.494582,1643417403153,8.728462,4.494582,4.0,1.0
579296,2019,China,Vietnam,97288421350,61875302462,197,180,2455821664459,0.001958,5.0,3.961543,1643417403153,3.765039,3.961543,5.0,8.0
579161,2019,China,Germany,88846081746,108326412325,99,84,2455821664459,0.000289,6.0,3.617774,1643417403153,6.591534,3.617774,6.0,4.0
579176,2019,China,India,73721380754,17305499689,49,56,2455821664459,0.000000,7.0,3.001903,1643417403153,1.053019,3.001903,7.0,25.0
579221,2019,China,Netherlands,67789491521,12684344221,166,130,2455821664459,0.000022,8.0,2.760359,1643417403153,0.771827,2.760359,8.0,29.0
579290,2019,China,United Kingdom,55728738053,29943601329,205,166,2455821664459,0.063316,9.0,2.269250,1643417403153,1.822033,2.269250,9.0,15.0
579273,2019,China,Taiwan,55367450265,111201418021,121,114,2455821664459,0.000000,10.0,2.254539,1643417403153,6.766474,2.254539,10.0,3.0


In [92]:
grouped_df.columns

Index(['year', 'location_name', 'partner_name', 'export_value', 'import_value',
       'num_export_partners_by_country_year',
       'num_import_partners_by_country_year', 'total_export_by_location_year',
       'share', 'rank'],
      dtype='object')

In [126]:
grouped_df[(grouped_df.location_name == 'Albania') & (grouped_df.year== 2019)].sort_values(by='export_value', ascending=False)

,year,location_name,partner_name,export_value,import_value,num_export_partners_by_country_year,num_import_partners_by_country_year,total_export_by_location_year,share,rank,proportion,total_import_by_location_year,proportion_import_year,proportion_export_year,rank_export_year,rank_import_year
574763,2019,Albania,Italy,1242824965,1517888777,119,101,2849791895,0.000000,1.0,43.611078,5697700835,26.640373,43.611078,1.0,1.0
574812,2019,Albania,Spain,217771516,109644247,189,139,2849791895,0.000012,2.0,7.641664,5697700835,1.924359,7.641664,2.0,8.0
574805,2019,Albania,Serbia,180822204,162215080,162,129,2849791895,0.000018,3.0,6.345102,5697700835,2.847027,6.345102,3.0,7.0
574749,2019,Albania,Germany,155890335,293856572,124,96,2849791895,0.008624,4.0,5.470236,5697700835,5.157459,5.470236,4.0,5.0
574751,2019,Albania,Greece,142482493,632845642,195,139,2849791895,0.000159,5.0,4.999751,5697700835,11.107035,4.999751,5.0,2.0
574746,2019,Albania,France,89243687,55376373,124,96,2849791895,0.000048,6.0,3.131586,5697700835,0.971907,3.131586,6.0,22.0
574730,2019,Albania,China,81943672,597300409,183,160,2849791895,0.000042,7.0,2.875427,5697700835,10.483183,2.875427,7.0,3.0
574789,2019,Albania,North Macedonia,77486382,79635661,55,58,2849791895,0.009362,8.0,2.719019,5697700835,1.397681,2.719019,8.0,16.0
574736,2019,Albania,Czech Republic,48977840,58326897,183,160,2849791895,0.000070,9.0,1.718646,5697700835,1.023692,1.718646,9.0,19.0
574782,2019,Albania,Montenegro,48326277,12849095,105,109,2849791895,0.001452,10.0,1.695783,5697700835,0.225514,1.695783,10.0,39.0


In [111]:
grouped_df.head()

,year,location_name,partner_name,export_value,import_value,num_export_partners_by_country_year,num_import_partners_by_country_year,total_export_by_location_year,share,rank,proportion
257328,2007,Afghanistan,Albania,0,2057713,36,36,372909932,0.000000,56.0,0.000000
360830,2011,Afghanistan,Albania,0,106445,36,36,523362598,0.004745,58.0,0.000000
17672,1996,Afghanistan,Algeria,212007,0,36,36,83146051,0.000000,29.0,0.254981
36008,1997,Afghanistan,Algeria,181929,0,36,36,103148983,0.190143,37.0,0.176375
55224,1998,Afghanistan,Algeria,383822,0,36,36,120012405,0.000000,30.0,0.319819


In [132]:
grouped_df[(grouped_df.partner_name == 'China') & (grouped_df.rank_import_year== 1)].sort_values(by='export_value', ascending=False)

,year,location_name,partner_name,export_value,import_value,num_export_partners_by_country_year,num_import_partners_by_country_year,total_export_by_location_year,share,rank,proportion,total_import_by_location_year,proportion_import_year,proportion_export_year,rank_export_year,rank_import_year
372804,2011,Japan,China,170958455179,153152700290,179,140,838150078665,9.589722e-05,1.0,20.397117,735164431165,20.832442,20.397117,1.0,1.0
569689,2018,South Korea,China,168902196787,108722511846,76,86,612430764370,0.000000e+00,1.0,27.578986,493199014356,22.044349,27.578986,1.0,1.0
346483,2010,Japan,China,157159817269,124243140389,179,140,780679476959,2.563710e-05,1.0,20.131158,591237761836,21.014074,20.131158,1.0,1.0
619152,2020,Taiwan,China,154785401983,60368859232,196,150,411290204811,1.018482e-02,1.0,37.634109,261787419129,23.060260,37.634109,1.0,1.0
399318,2012,Japan,China,152907382512,155058208308,179,140,792122959388,5.688766e-06,1.0,19.303491,747575969461,20.741465,19.303491,1.0,1.0
559991,2018,Japan,China,151778393958,148735131462,179,140,751574698361,2.192971e-04,1.0,20.194718,648296965622,22.942438,20.194718,1.0,1.0
435817,2013,South Korea,China,150744203385,89698705390,76,86,565025601845,4.318640e-03,1.0,26.679181,472562287292,18.981351,26.679181,1.0,1.0
462677,2014,South Korea,China,150454942703,98947285760,76,86,577729082991,0.000000e+00,1.0,26.042473,484534910624,20.421085,26.042473,1.0,1.0
610179,2020,Japan,China,148212859155,144353378082,179,140,651870678264,3.167677e-05,1.0,22.736543,552992791189,26.104025,22.736543,1.0,1.0
542828,2017,South Korea,China,148087751252,101859345164,76,86,571651192082,1.275161e-03,1.0,25.905264,445699134452,22.853835,25.905264,1.0,1.0


In [141]:
grouped_df[(grouped_df.partner_name == 'China') & (grouped_df.rank_import_year<4)& (grouped_df.proportion_import_year>14.999)]['year'].value_counts()

2020    109
2015     91
2019     91
2017     85
2016     81
2018     81
2014     60
2013     58
2012     56
2011     46
2010     39
2009     39
2007     29
2008     27
2006     23
2005     19
2004     16
2003     14
2002     13
2001     11
2000     10
1995      9
1999      9
1998      8
1996      6
1997      5
Name: year, dtype: int64

In [135]:
grouped_df.location_name.nunique()

242

In [139]:
grouped_df[(grouped_df.partner_name == 'United States of America') & (grouped_df.rank_import_year<4)]['year'].value_counts()

2001    101
2000     98
1998     96
1995     96
1997     94
1999     91
2004     90
1996     90
2002     88
2007     87
2006     84
2009     83
2015     83
2020     82
2005     82
2003     81
2016     81
2017     80
2019     80
2012     79
2014     78
2008     77
2011     76
2018     74
2013     73
2010     72
Name: year, dtype: int64